# Send REST requests to the WeatherApp.API running in Aspire

In [32]:
// First set the base url.
var baseUrl = "https://localhost:7220";

## Healthcheck 💉

In [ ]:
#!set --value @csharp:baseUrl --name baseUrl
GET {{baseUrl}}/

## GET Weather Forecast 🌦️🌈🔆

In [ ]:
#!set --value @csharp:baseUrl --name baseUrl
GET {{baseUrl}}/v1/weather-forecast/Bristol/{{$datetime iso8601}}

## POST Colected Weather Data 🌡️☔💨

In [39]:
// Generate a random reference...
string[] fruits = ["apple", "banana", "cherry", "date", "elderberry", "fig", "grape", "honeydew", "kiwi", "lemon", "mango", "nectarine", "orange", "papaya", "quince", "raspberry", "strawberry", "tangerine", "watermelon", "guava"];
var randomReference = $"{fruits[new Random().Next(fruits.Length)]}-{Random.Shared.Next(10, 99)}";
randomReference.Display();

lemon-94

In [40]:
#!set --value @csharp:baseUrl --name baseUrl
#!set --value @csharp:randomReference --name randomReference

POST {{baseUrl}}/v1/collected-weather-data/Bristol/{{randomReference}}
Content-Type: application/json
x-request-id: {{$guid}}
    
{
    "points": [
        {
            "time": "{{$datetime iso8601}}",
            "windSpeedInMetersPerSecond": {{$randomInt 0 69}}.{{$randomInt 0 99}},
            "windDirection": "N",
            "temperatureReadingInDegreesCelcius": {{$randomInt -15 40}}.{{$randomInt 0 99}},
            "humidityReadingInPercent": {{$randomInt 20 100}}
        }
    ]
}

Name,Value
X-Request-ID,18703c47-043a-4404-94bc-2f5ef0a72adf
traceparent,00-7d860568970c7afd12becdfa2fdd8eec-693519955912928f-00
Content-Type,application/json
Content-Length,280
Name,Value
Date,"Thu, 24 Jul 2025 13:38:20 GMT"
Server,Kestrel
Transfer-Encoding,chunked
Content-Type,application/json; charset=utf-8
Content-Length,75


<div style="display:block; height:10rem;"></div>

## GET Traces 🤖

In [44]:
using System.Net.Http;
using System.Text.Json;

public class TraceData
{
    public string Resource { get; set; } = string.Empty;
    public string Source { get; set; } = string.Empty;
    public string DisplayName { get; set; } = string.Empty;
    public string TraceId { get; set; } = string.Empty;
    public string SpanId { get; set; } = string.Empty;    
    public Dictionary<string, object> Tags { get; set; } = new();

    public override string ToString()
    {
        var thirdPart = string.Empty;
        if(DisplayName == "Domain Event Insertion")
            thirdPart = Tags["domain-event.eventclassName"].ToString().Split('.').LastOrDefault();

        if(DisplayName == "User Notification Sent")
            thirdPart = $"✉️ reference: {Tags["user-notification-event.reference"].ToString()}";

        if(DisplayName == "Outbox Item Insertion")
            thirdPart = $"📫 outbox id: {Tags["outbox-item.Id"].ToString()}";
                   
        return $"{Resource.PadRight(30, '-')}> {DisplayName.PadRight(30, '-')}> {thirdPart}";
    } 
        
}

var client = new HttpClient() { BaseAddress = new Uri("http://localhost:8000"), DefaultRequestHeaders = { { "X-Api-Key", "11119999" } } };
var response = await client.GetAsync("traces");
var responseJson = await response.Content.ReadAsStringAsync();
JsonSerializer.Deserialize<TraceData[]>(responseJson, JsonSerializerOptions.Web ).Display();

index value 0 NotificationService-----------> User Notification Sent--------> ✉️ reference: lemon-94 Resource NotificationService Source UserNotificationEventHandler DisplayName User Notification Sent TraceId 7d860568970c7afd12becdfa2fdd8eec SpanId 2c3e974b521ee8c4 Tags key value user-notification-event.body "Dear user, your data has been submitted and included in our latest model" user-notification-event.reference "lemon-94" user-notification-event.timestamp "2025-07-24T13:38:24.2388246+00:00" 1 WeatherApp.EventListener------> Outbox Item Insertion---------> 📫 outbox id: 18 Resource WeatherApp.EventListener Source OutboxRepository DisplayName Outbox Item Insertion TraceId 7d860568970c7afd12becdfa2fdd8eec SpanId 4211a66db730f4aa Tags key value outbox-item.TypeName "UserNotificationEvent" outbox-item.Id 18 2 WeatherApp.EventListener------> Domain Event Insertion--------> ModelingDataAccepted Resource WeatherApp.EventListener Source EventRepositorySql DisplayName Domain Event Insertion TraceId 7d860568970c7afd12becdfa2fdd8eec SpanId 1c82b031c2c50886 Tags key value domain-event.streamId "788fd554-1385-4878-ac76-a61810ea218c" domain-event.version "5" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.ModelingDataAccepted" 3 WeatherApp.API----------------> Domain Event Insertion--------> SubmittedToModeling Resource WeatherApp.API Source EventRepositorySql DisplayName Domain Event Insertion TraceId 7d860568970c7afd12becdfa2fdd8eec SpanId 8b97a1d70e1bdc94 Tags key value domain-event.streamId "788fd554-1385-4878-ac76-a61810ea218c" domain-event.version "4" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.SubmittedToModeling" 4 WeatherApp.API----------------> Domain Event Insertion--------> LocationIdFound Resource WeatherApp.API Source EventRepositorySql DisplayName Domain Event Insertion TraceId 7d860568970c7afd12becdfa2fdd8eec SpanId 17a2635aa7cdb8b8 Tags key value domain-event.streamId "788fd554-1385-4878-ac76-a61810ea218c" domain-event.version "2" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.LocationIdFound" 5 WeatherApp.EventListener------> Domain Event Insertion--------> ModelUpdated Resource WeatherApp.EventListener Source EventRepositorySql DisplayName Domain Event Insertion TraceId 7d860568970c7afd12becdfa2fdd8eec SpanId c3891b76ef86d0e6 Tags key value domain-event.streamId "788fd554-1385-4878-ac76-a61810ea218c" domain-event.version "8" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.ModelUpdated" 6 WeatherApp.EventListener------> Domain Event Insertion--------> SubmissionComplete Resource WeatherApp.EventListener Source EventRepositorySql DisplayName Domain Event Insertion TraceId 7d860568970c7afd12becdfa2fdd8eec SpanId cbd88233c29c672e Tags key value domain-event.streamId "788fd554-1385-4878-ac76-a61810ea218c" domain-event.version "7" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.SubmissionComplete" 7 WeatherApp.EventListener------> Domain Event Insertion--------> PendingContributorPaymentCommitted Resource WeatherApp.EventListener Source EventRepositorySql DisplayName Domain Event Insertion TraceId 7d860568970c7afd12becdfa2fdd8eec SpanId cee27b5e591b5f64 Tags key value domain-event.streamId "788fd554-1385-4878-ac76-a61810ea218c" domain-event.version "6" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.PendingContributorPaymentCommitted" 8 WeatherApp.API----------------> Domain Event Insertion--------> PendingContributorPaymentPosted Resource WeatherApp.API Source EventRepositorySql DisplayName Domain Event Insertion TraceId 7d860568970c7afd12becdfa2fdd8eec SpanId af008607e68b4e0e Tags key value domain-event.streamId "788fd554-1385-4878-ac76-a61810ea218c" domain-event.version "3" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.PendingContributorPaymentPosted" 9 WeatherApp.API----------------> Domain Event Insertion--------> WeatherDataCollectionInitiated Resource WeatherApp.API Source EventRepositorySql DisplayName Domain Event Insertion Tra